---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 
          'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 
          'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 
          'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 
          'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 
          'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 
          'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 
          'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 
          'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 
          'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 
          'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 
          'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    #----------------
    
    # Build Dataframe by row
    df = pd.read_csv('../materials/university_towns.txt', sep="\n", names = ['RegionName'])
    
    # Remove every character after " (".
    df['RegionName'] = df.RegionName.apply(lambda x: x.split('(')[0])
    
    # Build State column (np.nan when it's a town)
    df['State'] = df.RegionName.apply(lambda x: x[:-6] if x[-6:] == '[edit]' else np.nan)
    
    # Using filla to propagate the respective State to each Town
    df.State.fillna(method='ffill', inplace = True)
    
    # Remove the row corresponding to States
    df = df[df.RegionName.str.contains ('edit]') == False]
    
    # Remove the \n
    df['State'] = df['State'].str.strip() 
    df['RegionName'] = df['RegionName'].str.strip() 
    
    return df

get_list_of_university_towns()

,RegionName,State
1,Auburn,Alabama
2,Florence,Alabama
3,Jacksonville,Alabama
4,Livingston,Alabama
5,Montevallo,Alabama
6,Troy,Alabama
7,Tuscaloosa,Alabama
8,Tuskegee,Alabama
10,Fairbanks,Alaska
12,Flagstaff,Arizona


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    #--------------
    
    # Read from excel
    _df = pd.read_excel('../materials/gdplev.xls', 
                   skiprows=8, 
                   usecols = [4,6], 
                   names = ['Quarter',
                            'GDP in billions of chained 2009 dollars'])
    
    # Get values after 2000 only
    df = _df.iloc[221:,:]
    
    # Compute the difference between each row and the previous one 
    # and keep only the rows where it is negative
    df = df[df['GDP in billions of chained 2009 dollars'].diff() < 0]
    
    # Compute the difference between each index 
    df['Index']= df.index
    diff = df.Index.diff()
    
    # Get the first row where the difference between the indexes is 1 
    # We have to subtract 1 to have the first year of the pair
    ind = diff[diff == 1].index[0] - 1
    
    # Get the Quarter using the index
    return _df.Quarter.iloc[ind].encode('ascii', 'ignore')

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    #--------------
    
    # Read from excel
    _df = pd.read_excel('../materials/gdplev.xls', 
                   skiprows=8, 
                   usecols = [4,6], 
                   names = ['Quarter',
                            'GDP in billions of chained 2009 dollars'])
    
    # Get start
    start = get_recession_start()
    
    # Get Index
    index = _df[_df.Quarter == start].index[0]
    
    # Get values after 2000 only
    df = _df.iloc[index:,:]
    
    # Get differences
    df_diff = df['GDP in billions of chained 2009 dollars'].diff()
    
    # Get index as the first positive diference since the start
    ind = df_diff[df_diff >0].index[0] + 1
    
    return _df.Quarter.iloc[ind].encode('ascii', 'ignore')

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    #--------------
    
    # Read from excel
    _df = pd.read_excel('../materials/gdplev.xls', 
                   skiprows=8, 
                   usecols = [4,6], 
                   names = ['Quarter',
                            'GDP in billions of chained 2009 dollars'])
    
    # Get start
    end = get_recession_end()
    
    # Get Index
    index = _df[_df.Quarter == end].index[0]
    
    return _df.Quarter.iloc[index -2].encode('ascii', 'ignore')

get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    #----------------------------
    
    # Obtain list with quarter names
    quarters = ['q1', 'q2', 'q3', 'q4']
    years = [y for y in range(2000,2017)]
    columns = ['{}{}'.format(y,q) for y in years for q in quarters if y != 2016 or q != 'q4']
    
    # Load and drop columns until year 2000
    df = pd.read_csv('../materials/City_Zhvi_AllHomes.csv')
    df['State'] = df.State.map(states)
    df = (df.set_index(['State', 'RegionName'])
            .drop(set(df.columns[:51]).difference ({'State', 'RegionName'}), axis = 1))
    
    
    # Get the indexes corresponding to each quarter
    periods = [list(df.columns)[x:x+3] for x in range(0, len(list(df.columns)), 3)]
    
    # Build new columns
    for col,q in zip(columns, periods):
        df[col] = df[q].mean(axis=1)
     
    # Keep only new columns
    df = df.drop(df.columns[4:204], axis = 1)
        
    return df
     
convert_housing_data_to_quarters()

,,2000-01,2000-02,2000-03,2000-04,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,204400.0,207000.0,209800.0,212300.0,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,136800.0,138300.0,140100.0,141900.0,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,52700.0,53100.0,53200.0,53400.0,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,111000.0,111700.0,112800.0,113700.0,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,131700.0,132600.0,133500.0,134100.0,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,219200.0,222900.0,226600.0,230200.0,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,85100.0,84500.0,83800.0,83600.0,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,364100.0,374000.0,384700.0,395700.0,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Compute decline or growth of housing prices between the recession start and bottom
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    df = convert_housing_data_to_quarters()
    df.reset_index(inplace = True)
    df['diff'] = df[recession_start] - df[recession_bottom]
    
    # Divide populations
    university_towns = get_list_of_university_towns()['RegionName']
    df['is_university'] = df.RegionName.isin(university_towns)
    is_uni = df.loc[df.is_university == True, 'diff'].dropna()
    not_uni = df.loc[df.is_university == False, 'diff'].dropna()
    
    # Compute p_value, different and better
    p_value = list(ttest_ind(not_uni, is_uni))[1]
    different = True if p_value < 0.01 else False
    better = 'university town' if is_uni.mean() < not_uni.mean() else 'non-university town'
    
    return different, p_value, better

run_ttest()

(True, 7.1245161701732997e-06, 'university town')